This notebook will pull from the FPL API the weekly results for each player and stack all the results into a single df

Update weekly

In [9]:
import pandas as pd
import numpy as np
import requests

In [45]:
top500 = pd.read_csv('top500.csv', index_col='id_')
top500.columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'in_dreamteam',
       'influence', 'minutes', 'news', 'news_added', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'red_cards', 'saves', 'second_name', 'selected_by_percent', 'special',
       'squad_number', 'status', 'team', 'team_code', 'threat', 'total_points',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'web_name',
       'yellow_cards'],
      dtype='object')

In [28]:
histories = {}

ids = top500['id_']

for id_ in ids:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    
    histories[id_] = player_history

In [51]:
history_dfs = {}

for key in histories:
    df = pd.DataFrame(histories[key])
    df['gameweek'] = [i+1 for i in range((len(df.index)))]
    df['id_'] = key
    df['sum_points'] = [sum(df['total_points'].iloc[:i+1]) for i in range(len(df.index))]
    df['web_name'] = top500.loc[key]['web_name']
    history_dfs[key] = df
    
    
    

In [52]:
type(history_dfs)

dict

In [53]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.shape

(5860, 35)

In [54]:
top500histories.to_csv('top500histories.csv')

In [55]:
top500histories.head()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,gameweek,id_,sum_points,web_name
0,0,0,5,1,4.9,166,5,0,0,1.6,...,0,0,0,90,True,0,1,166,2,Vardy
1,0,0,-3,0,2.3,166,14,1,0,0.9,...,-82257,18334,100591,90,False,0,2,166,4,Vardy
2,0,2,25,0,1.8,166,27,1,1,6.2,...,-78612,32101,110713,89,False,0,3,166,12,Vardy
3,1,3,63,0,13.6,166,36,1,2,14.6,...,-7210,49194,56404,89,True,0,4,166,28,Vardy
4,0,0,5,0,11.3,166,45,1,0,1.7,...,83507,136063,52556,89,False,0,5,166,30,Vardy
